# TP5 - NLP

Mục đích của TP này : dự đoán cột target là career level của vị trí tuyển dụng thông qua các cột features khác của một job description

In [1]:
# Check the directory now where i'm working
import os
print("Working directory:", os.getcwd())

Working directory: c:\Users\KTANG\Downloads\Python\Learn DS\TPs\TP5 - NLP


### Importing library


In [26]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


### B1 : Data collection : phải đi làm mới có (bỏ qua)
### Bước Importing data

In [3]:
df = pd.read_excel("final_project.ods", engine="odf", dtype=str)
df

,title,location,description,function,industry,career_level
0,Technical Professional Lead - Process,"Houston, TX","Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager
1,Cnslt - Systems Eng- Midrange 1,"Seattle, WA","Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager
2,SharePoint Developers and Solution Architects,"Dallas, TX",We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager
3,Business Information Services - Strategic Acco...,North Carolina,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager
4,Strategic Development Director (procurement),"Austin, TX",Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter
...,...,...,...,...,...,...
8069,Sales Engineer,"Waltham, MA",You are the primary technical resource for the...,sales,Hardware,senior_specialist_or_project_manager
8070,"Director of Sales, ClosetMaid","Atlanta, GA",Act as the primary contact for Home Depot deci...,sales,Furniture,senior_specialist_or_project_manager
8071,PROPERTY MANAGEMENT: Regional Vice President,"Los Angeles, CA","Responsible for the management of office, mult...",finance_accounting,Real Estate Investment Companies,manager_team_leader
8072,Technical Manager SAP Senior,New York,Viseo Business Solution Practice invested in S...,consulting,IT Consulting,senior_specialist_or_project_manager


In [4]:
df.info(); df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8074 entries, 0 to 8073
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         8074 non-null   object
 1   location      8074 non-null   object
 2   description   8073 non-null   object
 3   function      8074 non-null   object
 4   industry      8074 non-null   object
 5   career_level  8074 non-null   object
dtypes: object(6)
memory usage: 378.6+ KB


,title,location,description,function,industry,career_level
count,8074,8074,8073,8074,8074,8074
unique,6790,1081,7973,19,352,6
top,Account Manager,"New York City, NY",Practice Medicine in Northern California with ...,sales,Software Companies,senior_specialist_or_project_manager
freq,58,401,5,3151,688,4338


Vì cột "description" có đúng một hàng bị NULL, ta không cần fill mà drop hàng đó luôn

In [5]:
df = df.dropna(axis=0, how='any')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8073 entries, 0 to 8073
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         8073 non-null   object
 1   location      8073 non-null   object
 2   description   8073 non-null   object
 3   function      8073 non-null   object
 4   industry      8073 non-null   object
 5   career_level  8073 non-null   object
dtypes: object(6)
memory usage: 441.5+ KB


### Bước chỉnh sửa cột "location"

<ins>Mục đích :</ins> để đến bước tiền xử lý văn bản ta sẽ xử lý nhẹ nhàng hơn (nếu không thì sẽ gặp vấn đề như khi làm OneHot bên dưới), khi đó ta sẽ sử dụng các thuật toán TF-IDF hoặc OneHot để chuyển đổi văn bản.

<ins>Vấn đề sau OneHot :</ins>
- Có tận 963 cột được tạo ra sau khi dùng OneHotEncoder, quá nhiều và gây khó khăn cho việc xử lý dữ liệu. Nên ta lập luận rằng để dự đoán career level thì các thành phố trong cùng bang sẽ như nhau, nên ta không cần biết thông tin về từng thành phố, mà chỉ cần biết thông tin về tiểu bang thôi

<ins>Hướng làm để tách các giá trị tiểu bang của từng hàng :</ins>
- Cột nay có dữ liệu không sạch, giá trị tiểu bang có thể sau dấu phẩy hoặc không, có thể viết tắt hoặc không, nên ta sẽ sử dụng **RegEx** để tách chúng một cách tốt nhất
- Sử dụng các trang regex trên mạng để test các parameter của **RegEx** cần dùng
- Viết một hàm để tìm và trả về các giá trị của tiểu bang trong mỗi hàng vào một list => Dùng **findall** trong thư viên `re`

<ins>Cách làm :</ins>
- Viết hàm `filter_state` để tách các giá trị tiểu bang trong mỗi hàng ngay sau bước Importing Data

<ins>Kết quả :</ins>

- Sau khi dùng function **filter_state** trước khi dùng **OneHotEncoder**, đã giảm cột xuống còn 95 cột thành vì 963 cột, rất hợp lý và dễ dàng xử lý hơn

In [6]:
def filter_state(location_value):
    state = re.findall("\,\s[A-Z]{2}$", location_value)
    if len(state) > 0:
        return state[0][2:4]
    else:
        return location_value

<>:2: SyntaxWarning: invalid escape sequence '\,'
<>:2: SyntaxWarning: invalid escape sequence '\,'
C:\Users\KTANG\AppData\Local\Temp\ipykernel_18900\1869176104.py:2: SyntaxWarning: invalid escape sequence '\,'
  state = re.findall("\,\s[A-Z]{2}$", location_value)


Dùng hàm `apply()` để áp dụng function ``filter_state vào từng hàng của cột dữ liệu (Giải thích phía dưới ở phần làm feature thứ hai "location")

In [7]:
df["location"] = df["location"].apply(filter_state)

C:\Users\KTANG\AppData\Local\Temp\ipykernel_18900\3792979346.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["location"] = df["location"].apply(filter_state)


### Bước chia dữ liệu (trước bước tiền xử lý dữ liệu):

Là bước chia dữ liệu theo :
- Phần A: theo chiều dọc (theo cột), tức là chia các cột X (các cột feature) với cột Y (cột target)
- Phần B: theo chiều ngang, tức là chia datasets thành 3 dataset con gồm bộ train, bộ validation và bộ test theo tỷ lệ tùy chọn (8-1-1, 9-0.5-0.5, 6-2-2)


Phần A

In [8]:
# Xác định target
target = "career_level"

x = df.drop(target, axis=1)
y = df[target]

Phần B

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state=42, stratify=y)

print(f" Shape of train set is {x_train.shape} on x and {y_train.shape} on y")
print(f" Shape of test set is {x_test.shape} on x and {y_test.shape} on y")
print("\n")
print(f"- Nb of samples of train set is {y_train.value_counts()} \n\n- And of test set is {y_test.value_counts()}")

 Shape of train set is (6458, 5) on x and (6458,) on y
 Shape of test set is (1615, 5) on x and (1615,) on y


- Nb of samples of train set is career_level
senior_specialist_or_project_manager      3469
manager_team_leader                       2138
bereichsleiter                             768
director_business_unit_leader               56
specialist                                  24
managing_director_small_medium_company       3
Name: count, dtype: int64 

- And of test set is career_level
senior_specialist_or_project_manager      868
manager_team_leader                       534
bereichsleiter                            192
director_business_unit_leader              14
specialist                                  6
managing_director_small_medium_company      1
Name: count, dtype: int64


### Bước  tiền xử lý dữ liệu chuỗi string:

![Text Preprocessing](../Text_preprocessing.png)

<ins>*Lúc nào dùng TF-IDF, lúc nào dùng OneHot :*</ins> Kiểm tra số lượng giá trị riêng biệt trong một cột, nếu nó ít hơn 100 thì dùng OneHotEncoder, nếu nó nhiều hơn thì dùng TF-IDF

<ins>*Cách làm :*</ins> Sử dụng thẳng `ColumnTransformer` để biến đổi tuần tự 5 cột features gồm `title`, `location`, `description`, `function`, `industry` (ở đây không dùng **Pipeline** nữa vì trong NLP không cần bước Imputation chuẩn như trong ML Supervised hay Unsupervised, do đó chỉ còn một bước xử lý nên không cần dùng **Pipeline** nữa) 

<ins>*Lưu ý :*</ins> Khi sử dụng `TfidfVectorizer` trong `ColumnTransformer`, ta không được gộp tất cả các cột vào một `TfidfVectorizer` mà phải gán riêng cho từng bước cho từng cột (mặc định, nếu không sẽ lỗi), trong khi OneHot Encoder thì có thể gộp tất cả các cột vào một bước (tuy nhiên ở đây tôi vẫn tách ra để cho dễ nhìn và dễ apply các bước khác vào từng cột về sau).

In [ ]:
# Kiểm tra các giá trị riêng biệt của từng feature để quyết định xem nên áp dụng TF- IDF hay One Hot Encoding vào feature đó 
# Nhắc lại : >100 giá trị riêng biệt thì dùng TF-IDF, =<100 thì dùng One Hot Encoding

for col in x_train.columns:
    num_unique = df[col].nunique() # Tính số giá trị riêng biệt của mỗi feature, giống len(df[col].unique())
    if num_unique > 100:
        print(f"- {col} has {num_unique} unique value → Must use TF-IDF")
    else:
        print(f"- {col} has {num_unique} unique value → Must use One Hot Encoding")

- title has 6789 unique value → Must use TF-IDF
- location has 97 unique value → Must use One Hot Encoding
- description has 7973 unique value → Must use TF-IDF
- function has 19 unique value → Must use One Hot Encoding
- industry has 352 unique value → Must use TF-IDF


In [ ]:
text_preprocessor = ColumnTransformer(transformers=[ 
    ("title_col", TfidfVectorizer(stop_words="english", ngram_range=(1,1)), "title"),
    ("location_col", OneHotEncoder(sparse_output=False), ["location"]),
    ("description_col", TfidfVectorizer(stop_words="english", ngram_range=(1,2)), "description"),
    ("function_col", OneHotEncoder(sparse_output=False), ["function"]),
    ("industry_col", TfidfVectorizer(stop_words="english", ngram_range=(1,1)), "industry")])

text_preprocessor 

ColumnTransformer(transformers=[('title_col',
                                 TfidfVectorizer(stop_words='english'),
                                 'title'),
                                ('location_col',
                                 OneHotEncoder(sparse_output=False),
                                 'location'),
                                ('description_col',
                                 TfidfVectorizer(ngram_range=(1, 2),
                                                 stop_words='english'),
                                 'title'),
                                ('function_col',
                                 OneHotEncoder(sparse_output=False),
                                 'function'),
                                ('industry_col',
                                 TfidfVectorizer(stop_words='english'),
                                 'industry')])

#### Feature đầu tiên : "title"

Bây giờ ta có các cột feature dạng chuỗi văn bản, ta cần chuyển chúng thành dạng số để có thể sử dụng trong mô hình học máy. Ta sử dụng thuật toán TF-IDF để chuyển đổi chúng về dạng các vector. 

In [10]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,1))
processed_data_title = vectorizer.fit_transform(x_train["title"])

print(vectorizer.get_feature_names_out())
print(processed_data_title.toarray())

['00' '014304' '01ef' ... 'zpic' 'zv' 'zynga']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


#### Feature thứ hai : "location"

- Cách 1 : dùng TF-IDF (TfidfVectorizer)
- Cách 2 : tách ra thành cột thành phố và cột tiểu bang rồi dùng OneHotEncoder
- Cách 3 : dùng thẳng OneHotEncoder (dùng cách này ở đây)

<ins>Vấn đề sau OneHot :</ins>
- Có tận 963 cột được tạo ra sau khi dùng OneHotEncoder, quá nhiều và gây khó khăn cho việc xử lý dữ liệu. Nên ta lập luận rằng để dự đoán career level thì các thành phố trong cùng bang sẽ như nhau, nên ta không cần biết thông tin về từng thành phố, mà chỉ cần biết thông tin về tiểu bang thôi

<ins>Hướng làm để tách các giá trị tiểu bang của từng hàng :</ins>
- Cột nay có dữ liệu không sạch, giá trị tiểu bang có thể sau dấu phẩy hoặc không, có thể viết tắt hoặc không, nên ta sẽ sử dụng **RegEx** để tách chúng một cách tốt nhất
- Sử dụng các trang regex trên mạng để test các parameter của **RegEx** cần dùng
- Viết một hàm để tìm và trả về các giá trị của tiểu bang trong mỗi hàng vào một list => Dùng **findall** trong thư viên `re`

<ins>Cách làm :</ins>
- Viết hàm `filter_state` để tách các giá trị tiểu bang trong mỗi hàng ngay sau bước Importing Data

<ins>Kết quả :</ins>

- Sau khi dùng function **filter_state** trước khi dùng **OneHotEncoder**, đã giảm cột xuống còn 95 cột thành vì 963 cột, rất hợp lý và dễ dàng xử lý hơn

In [11]:
encoder = OneHotEncoder(sparse_output=False)
processed_data_location = encoder.fit_transform(x_train[["location"]])
processed_data_location.shape

(6458, 95)

#### Feature thứ ba : "description"

Ta sử dụng thuật toán TF-IDF để chuyển đổi chúng về dạng các vector. 

In [12]:
vectorizer_bigram = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
processed_data_description = vectorizer_bigram.fit_transform(x_train["description"])
processed_data_description

print(len(vectorizer_bigram.vocabulary_)) # Thay vì nếu dùng ngram_range = (1,1) thì chỉ có 66675 token thôi

847124


#### Feature thứ tư : "function"

Sau khi kiểm tra số lượng giá trị riêng biệt, ta sử dụng thẳng thuật toán OneHotEncoder để chuyển đổi chúng về dạng các vector. 

In [25]:
# Kiểm tra số lượng giá trị riêng biệt trong một cột, nếu nó ít hơn 100 thì dùng OneHotEncoder, 
# nếu nó nhiều hơn thì dùng TF-IDF

len(df["function"].unique())

19

#### Feature thứ tư : "industry"

Sau khi kiểm tra số lượng giá trị riêng biệt, ta sử dụng thẳng thuật toán TF-IDF để chuyển đổi chúng về dạng các vector. 

In [24]:
# Kiểm tra số lượng giá trị riêng biệt trong một cột, nếu nó ít hơn 100 thì dùng OneHotEncoder, 
# nếu nó nhiều hơn thì dùng TF-IDF

len(df["industry"].unique())

352